<a href="https://colab.research.google.com/github/bilmark0/Agile-Manufacturing-TDK-/blob/main/CV/CNN_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade wandb

In [2]:
import os
import shutil
import random
from google.colab import drive
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import wandb
import PIL
import PIL.Image
import sys
from google.colab import files

In [3]:
# Upload the kaggle.json file
uploaded = files.upload()

# Move the uploaded file to the .kaggle directory
kaggle_api_path = os.path.expanduser("~/.kaggle")
if not os.path.exists(kaggle_api_path):
    os.makedirs(kaggle_api_path)

# Ensure the file is set with proper permissions
kaggle_json_path = next(iter(uploaded))  # Get the uploaded filename
os.rename(kaggle_json_path, f"{kaggle_api_path}/kaggle.json")
os.chmod(f"{kaggle_api_path}/kaggle.json", 0o600)

# Download the dataset from Kaggle
!kaggle datasets download -d markbilszky/agile-manufacturing-tdk --unzip

print("Dataset downloaded successfully.")

Saving kaggle (2).json to kaggle (2).json
Dataset URL: https://www.kaggle.com/datasets/markbilszky/agile-manufacturing-tdk
License(s): unknown
 99% 1.28G/1.29G [00:16<00:00, 80.2MB/s]
100% 1.29G/1.29G [00:16<00:00, 85.4MB/s]
Dataset downloaded successfully.


In [4]:
# Set your paths here
base_path = './'  # Replace with the directory containing Reference, error_2, and error_3
new_folder_path = os.path.join(base_path, 'training_data')

# Create the new folder
os.makedirs(new_folder_path, exist_ok=True)

# Move the folders
folders_to_move = ['Reference', 'error_2', 'error_3']
for folder_name in folders_to_move:
    shutil.move(os.path.join(base_path, folder_name), new_folder_path)

print("Folders moved successfully!")


Folders moved successfully!


In [5]:
# Function to get WandB API token by uploading a file
def get_wandb_token():
    uploaded = files.upload()  # Prompt user to upload a file
    for filename in uploaded.keys():
        with open(filename, 'r') as file:
            token = file.read().strip()
        return token
    return None
# Get WandB API token
wandb_token = get_wandb_token()

Saving wandb.txt to wandb.txt


In [ ]:
if wandb_token:
    wandb.login(key=wandb_token)
    wandb.init(project='your_project_name')

    data_dir = './training_data'
    img_height, img_width = 240, 380

    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.4,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.1
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary',
        subset='validation'
    )

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    base_model.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.00001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(
        'best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    )

    # Train the model and log metrics to W&B
    epochs = 50
    for epoch in range(epochs):
        history = model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=1,
            callbacks=[early_stopping, model_checkpoint]  # No W&B callback here
        )

        # Log metrics to W&B
        wandb.log({
            "epoch": epoch + 1,
            "loss": history.history['loss'][0],
            "val_loss": history.history['val_loss'][0],
            "accuracy": history.history['accuracy'][0],
            "val_accuracy": history.history['val_accuracy'][0]
        })

    # Evaluate the model
    loss, accuracy = model.evaluate(val_generator)
    print(f'Validation Loss: {loss}, Accuracy: {accuracy}')
else:
    print("Failed to retrieve the WandB API Token.")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: biebelbotond (biebelbotond-budapesti-m-szaki-s-gazdas-gtudom-nyi-egyetem). Use `wandb login --relogin` to force relogin


Found 11600 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 72/363 ━━━━━━━━━━━━━━━━━━━━ 59:42 12s/step - accuracy: 0.5507 - loss: 0.0571